### How to deploy and fine-tune DeepSeek models on AWS
> https://huggingface.co/blog/deepseek-r1-aws

In [1]:
!pip install sagemaker --upgrade
# !pip install --force-reinstall --no-cache-dir sagemaker==2.235.2


  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 99.8 MB/s eta 0:00:00
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Successfully uninstalled attrs-23.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.245.0
    Uninstalling sagemaker-2.245.0:
      Successfully uninstalled sagemaker-2.245.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sagemaker]/2 [sagemaker]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker-studio-analytics-extension 0.2.0 requires sparkmagic==0.22.0, but you have sparkmagic 0.21.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.3 which is incompatible.


In [2]:
import json
import sagemaker
import boto3
import time
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri


try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:


model_id = "deepseek-ai/deepseek-llm-r1-distill-qwen-1-5b"
model_name = model_id.split("/")[-1].lower()


# Hub Model configuration. https://huggingface.co/models
vllm_config = {
    "HF_MODEL_ID": model_name,
    "SM_NUM_GPUS": json.dumps(8)
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=get_huggingface_llm_image_uri("huggingface", version="3.0.1"),
    env=vllm_config,
    role=role,
)


In [9]:
# Generate a unique name with timestamp
# timestamp = int(time.time())
# endpoint_config_name = f"my-endpoint-config-{timestamp}"


endpoint_name = f"{model_name}-ep"

INSTANCE_TYPE = "ml.g6.2xlarge"
# The g6.2xlarge instance is in the GPU instance family with 8 vCPUs, 32 GiB of memory and up to 10 Gibps of bandwidth starting at $0.9776 per hour.
# https://instances.vantage.sh/aws/ec2/g6.2xlarge?currency=USD

# INSTANCE_TYPE = "ml.g5.2xlarge" 
# #The g5.2xlarge instance is in the GPU instance family with 8 vCPUs, 32 GiB of memory and up to 10 Gibps of bandwidth starting at $1.212 per hour.
# #https://instances.vantage.sh/aws/ec2/g5.2xlarge?currency=USD

# INSTANCE_TYPE = "ml.g5.12xlarge"
# # The g5. 12xlarge instance is in the GPU instance family with 48 vCPUs, 192 GiB of memory and 40 Gibps of bandwidth starting at $5.672 per hour.
# # https://instances.vantage.sh/aws/ec2/g5.12xlarge?currency=USD




# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    container_startup_health_check_timeout=2400, #1600
    endpoint_name=endpoint_name,
)



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 INSTANCE_TYPE = "ml.g6.2xlarge"                                                             │
│    3                                                                                             │
│    4 # deploy model to SageMaker Inference                                                       │
│ ❱  5 predictor = huggingface_model.deploy(                                                       │
│    6 │   endpoint_name=endpoint_name,                                                            │
│    7 │   initial_instance_count=1,                                                               │
│    8 │   instance_type=INSTANCE_TYPE,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/huggingface/model.py:326 in deploy             │
│                                                                                                  │
│   323 │   │   │   │   inference_tool=inference_tool,                                             │
│   324 │   │   │   )                                                                              │
│   325 │   │                                                                                      │
│ ❱ 326 │   │   return super(HuggingFaceModel, self).deploy(                                       │
│   327 │   │   │   initial_instance_count,                                                        │
│   328 │   │   │   instance_type,                                                                 │
│   329 │   │   │   serializer,                                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/model.py:1814 in deploy                        │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/session.py:6031 in                             │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   6028 │   │   │   config_options["ExecutionRoleArn"] = role                                     │
│   6029 │   │                                                                                     │
│   6030 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│ ❱ 6031 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6032 │   │                                                                                     │
│   6033 │   │   return self.create_endpoint(                

In [10]:
# send request
predictor.predict({"inputs": "What is the meaning of life?"})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # send request                                                                               │
│ ❱ 2 predictor.predict({"inputs": "What is the meaning of life?"})                                │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictor' is not defined

In [11]:
# Make sure you delete the endpoint once you finished testing it.
predictor.delete_model()
predictor.delete_endpoint()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Make sure you delete the endpoint once you finished testing it.                            │
│ ❱ 2 predictor.delete_model()                                                                     │
│   3 predictor.delete_endpoint()                                                                  │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictor' is not defined